# Oscillating wave 2D
## Test case description
For a description of the test cases and their setup, please refer to the preprint/paper.

## Parameter and error metric explanation
Mandatory and optional data columns for the CSV files providing the plot data. See the repository `README.md`
for general syntax information.

| column name | description | required |
| :-          | :-          | :-       |
| `solver` | solver name, e.g. *interFoam* | yes |
| `fluid_pairing` | name the two phases used for simulation, e.g. water-air | yes |
| `resolution` | mesh resolution as number of cells per spatial direction | yes |
| `time` | physical time of the data point in seconds | yes |
| `amplitude_at_center` | See paper | yes |

## Usage
The notebook is prepared such that you can just hit `Kernel` ->
`Restart & Run All`. This plots the temporal evolution of the amplitude centre x-coordinate for all fluid/fluid pairings, resolutions and solvers, as well as the accumulated amplitude error.  
Per default, all solvers for which data is available are added to each plot. If you want to compare a specific set of solvers, remove the `'all'` entry from `solvers` below and list the solvers you want to compare, e.g.
`solvers = ['interFoam', 'Fluent']`. If you prescribe a solver subset and a solver is missing in the plots, make sure that its spelling matches the entries in the CSV file exactly.  
The path stored in `init_comparison_dir` contains data obtained with the solver interIsoFoam for an water-air system. Different methods for setting the intial volume fraction have been used to demonstrate the initialization's influence on the temporal evolution. So data their is not intended for a comparison between solvers, but for volume fraction initalization methods.

In [ ]:
# Data directory containing CSV files
directory = 'data/hydrodynamic/oscillatingWave2D'
init_comparison_dir = 'data/hydrodynamic/oscillatingWave2D/volumeFractionInitComparison'

# Columns to read: index columns are used to addess data, data columns contain actual data
index_columns = ['solver', 'fluid_pairing', 'resolution']
data_columns = ['time', 'amplitude_at_center']

# Solver selection
solvers = ['all']

# Initial amplitude at time t=0
initial_amplitude = 5.5e-5

import math
import os

# Use all CSV files in the data directory
data_files = [filename for filename in os.listdir(directory) if 'csv' in filename and not 'reference' in filename]

def compute_difference_to_reference(df, a0):
    # This can be probably done much more elegant and concise manner
    df['delta'] = 0.0
    df['abs_delta'] = 0.0
    df['rel_abs_delta'] = 0.0    

    # Generate name to index map for columns
    cmap = dict()
    for cname in df.columns:
        cmap[cname] = df.columns.get_loc(cname)
    
    for ridx in df.index:
        time = df.iloc[ridx, cmap['time']]
        a_sol = df.iloc[ridx, cmap['amplitude_at_center']]
        if df.iloc[ridx, cmap['fluid_pairing']] == 'water-air':
            df.iloc[ridx, cmap['delta']] = a_sol - a0*math.exp(-11.4565*time)*math.cos(1012.4*time)
        elif df.iloc[ridx, cmap['fluid_pairing']] == 'gearoil-air':
            df.iloc[ridx, cmap['delta']] = a_sol - a0*math.exp(-181.052*time)*math.cos(7.92849e-3*time)
        elif df.iloc[ridx, cmap['fluid_pairing']] == 'oil_novec7500-water':
            df.iloc[ridx, cmap['delta']] = a_sol - a0*math.exp(-18.8965*time)*math.cos(499.691*time)
        else:
            sys.exit("Error: unknown fluid_pairing '", df.iloc[ridx, fp_cidx],"'.")
        df.iloc[ridx, cmap['abs_delta']] = math.fabs(df.iloc[ridx, cmap['delta']])
        df.iloc[ridx, cmap['rel_abs_delta']] = math.fabs(df.iloc[ridx, cmap['delta']])/a0
    
    return df

def create_time_evolution_plots(df, ref_df, single_plot_parameter, group_by_parameters,
                                file_name_addendum='', tmax=None, markers=None, lines=None):
    for params, subsetdf in  df.groupby(level=group_by_parameters):
        fig, ax = plt.subplots(figsize=figureSize, dpi=DPI)

        for value, plotdf in subsetdf.groupby(level=single_plot_parameter):
            ax.plot(plotdf["time"], plotdf["amplitude_at_center"],
                    label=value,
                    marker=markers[value], markevery=int(len(plotdf.index)/n_marks),
                    linestyle=lines[value])

        # Plot reference solution
        reference = ref_df[ref_df["fluid_pairing"]  == params[0]]
        ax.plot(reference["time"], reference["amplitude_at_center"], label="Reference")

        # Enforce upper and lower bounds for y axis limits
        ymin, ymax = ax.get_ylim()
        ymin = max(ymin, -0.7e-4)
        ymax = min(ymax, 0.7e-4)
        ax.set_ylim(ymin, ymax)
        
        # limit x-axis to the prescribed limit
        if tmax:
            xmin, xmax = ax.get_xlim()
            xmax = tmax
            ax.set_xlim(xmin, xmax)

        # Formatting
        ax.ticklabel_format(axis='y', style='scientific', scilimits=(-4,-4))
        ax.tick_params(axis='both', labelsize=plotfontsize)

        ax.set_ylabel("amplitude at center [m]", fontsize=plotfontsize)
        ax.set_xlabel("time [s]", fontsize=plotfontsize)

        ax.grid(which="both")
        ax.legend(fontsize=plotfontsize)

        title = title_from_parameters(group_by_parameters, params, "oscillating wave 2D")

        figure_directory = "figures"
        if not os.path.isdir(figure_directory):
            os.mkdir(figure_directory)
        fig.savefig(os.path.join(figure_directory,"oscillatingWave2D_" + format_for_filename(title) +
                  file_name_addendum + figure_suffix),
                  bbox_inches="tight")

        ax.set_title(title, fontsize=plotfontsize)

In [ ]:
from shared_functions import *
import sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update(latex_settings)

lin_df = solver_data_table(data_files, list(index_columns)+list(data_columns), data_dir=directory,
                           solver_subset=solvers)
# Ignore finest resolution n=256 for now
lin_df = lin_df.drop(lin_df[lin_df['resolution'] == 256].index)
lin_df.reset_index(drop=True, inplace=True)

df = compute_difference_to_reference(lin_df, initial_amplitude)

# In case of many notebook executions, e.g. for debugging:
# temporarily store dataframe with deltas and read it.
# The delta computation takes quite some time
#df.to_csv(directory + 'delta_frame.csv', index=False)
#df = pd.read_csv(directory + 'delta_frame.csv')

# Set multiindex
df.set_index(index_columns, inplace=True)
df.sort_index(inplace=True)

# Read reference solution
ref_df = pd.read_csv(os.path.join(directory, "reference_2D_oscillating_wave.csv"), sep=',', header=0)
# Convert column names to lowercase for consistency
ref_df.columns = ref_df.columns.str.lower()

# Each parameter combination is used to create a separate figure
group_by_parameters = ["fluid_pairing", "resolution"]

# Marker and line styles
n_marks = 10
solvers = list(df.index.unique(level='solver'))
solver_marker, solver_lines = assign_styles_to_solvers(solvers)


# Create Plots comparing solvers to each other
create_time_evolution_plots(df, ref_df, 'solver', group_by_parameters, markers=solver_marker,
                            lines=solver_lines)

# Create accumulated / time-averaged plots
plot_accumulated_errors(df, "oscillating_wave_2D", "rel_abs_delta", ylabel="time avg. rel. amplitude error",
                       solver_marker=solver_marker, file_name_addendum='_accumulated', time_rel=True)

# Create plots showing the influence of volume fraction initialization errors
data_files = [filename for filename in os.listdir(init_comparison_dir) if 'csv' in filename]
index_columns.append('init_approach')
df = read_solver_data(data_files, index_columns, data_columns, data_dir=init_comparison_dir)
init_approaches = list(df.index.unique(level='init_approach'))
init_marker, init_lines = assign_styles_to_solvers(init_approaches)
init_marker[init_approaches[0]] = '.' # Small markers here
create_time_evolution_plots(df, ref_df, 'init_approach', group_by_parameters,
                            file_name_addendum='_initialization_', tmax=0.01,
                            markers=init_marker, lines=init_lines
                            )